This code just glues together a bunch of things we've discussed in class. The one trick is taking the owner sample. 

## 1. Connect to GBQ

In [1]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
import random

In [2]:
# These first two values will be different on your machine. 
service_path = "C:\\users\\jchan\\dropbox\\teaching\\"
service_file = 'UMT-MSBA-7b4265df0ca4.json' # this is your authentication information  
gbq_proj_id = 'umt-msba'  # change this to your project_id
gbq_dataset_id = 'wedge_transactions' # and change this to your data set ID

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## 2. List of Owners

In [3]:
query = (
    "SELECT "
    "DISTINCT(card_no) "
    "FROM "
    "`umt-msba.wedge_transactions.transArchive_*` "
    "WHERE card_no != 3.0"
)

In [4]:
# And we execute queries with `client.query`
query_job = client.query(
    query,
    location="US",
)

In [5]:
all_owners = []

for row in query_job :
    all_owners.append(row[0])


In [6]:
len(all_owners)

23683

## 3. Sample

In [7]:
owner_sample = random.sample(all_owners,300)

## 4. Extract records

In [8]:
query = (
    "SELECT "
    "* "
    "FROM "
    "`umt-msba.wedge_transactions.transArchive_*` "
    "WHERE "
    "card_no IN("
)

query = query + ",".join([str(num) for num in owner_sample]) + ")"

In [9]:
# And we execute queries with `client.query`
query_job = client.query(
    query,
    location="US",
)

In [10]:
query_job.result()

## 5. Write to Text

In [11]:
headers = """datetime	register_no	emp_no	trans_no	upc	description	
             trans_type	trans_subtype	trans_status	department	
             quantity	Scale	cost	unitPrice	total	
             regPrice	altPrice	tax	taxexempt	foodstamp	
             wicable	discount	memDiscount	discountable	
             discounttype	voided	percentDiscount	ItemQtty	
             volDiscType	volume	VolSpecial	mixMatch	
             matched	memType	staff	numflag	itemstatus	
             tenderstatus	charflag	varflag	batchHeaderID	
             local	organic	display	receipt	card_no	store	
             branch	match_id	trans_id""".split()

In [12]:
with open("owner_sample.txt",'w') as ofile :
    ofile.write("\t".join(headers) + "\n")
    
    for idx, row in enumerate(query_job) :
        ofile.write("\t".join([str(element) for element in row]) + "\n")
    
#        if idx == 100 :
#            break
